In [65]:
import pandas as pd
import numpy as np

In [66]:
cardit_df = pd.read_excel('Sample_Data_from_IODA_v4 (China Post).xlsx', sheet_name='gls mail cardit')

In [67]:
event_df = pd.read_excel('Sample_Data_from_IODA_v4 (China Post).xlsx', sheet_name='cardit receptacle event')

In [68]:
master_df = pd.read_excel('Sample_Data_from_IODA_v4 (China Post).xlsx', sheet_name='awb master')

## Data Sanitisation

### Cardit Table Sanitisation

In [69]:
"""
Keep China Post as post office only
Keep first leg only
"""

cardit_df = cardit_df[cardit_df['post_office'] == 'China Post']
cardit_df = cardit_df[cardit_df['leg_number'] == 1]

In [70]:
# Filtering fields to keep 
columns_to_keep = ["dim_cgo_cargo_awb_master_sk", "receptacle_charge_weight", "shipment_origin", "shipment_dest", "fct_cgo_gls_mail_cardit_sk"]

cardit_df = cardit_df[columns_to_keep]

### Master Table Sanitisation

In [71]:
# Filtering fields to keep 
columns_to_keep = ["dim_cgo_cargo_awb_master_sk", "awb_number"]

master_df = master_df[columns_to_keep]

### Event Table Sanitisation

In [72]:
# Keep necessary fields first for easier processing
columns_to_keep = ["receptacle_id", "leg_number", "carrier_code", "flight_number", "flight_date", "shipment_origin", "shipment_dest", "actual_depart_datetime_local",	"actual_arrive_datetime_local", "fct_cgo_gls_mail_cardit_sk", 'uld_id', 'mail_consign_number']

event_df = event_df[columns_to_keep]

In [ ]:
# Remove possible "[Null]" string values
event_df = event_df.replace("[Null]", np.nan)

In [ ]:
# Remove rows where actual_depart_datetime_local is missing
# This will remove flights that have never departed
event_df = event_df[event_df['actual_depart_datetime_local'].notna()]

'\nThis will remove flights that have never departed\n'

In [75]:
"""
The same receptacle might have multiple actual_depart_datetime_local. We want to keep the latest departure only
"""

# Keep only the row with the latest actual_depart_datetime_local for each receptacle_id
event_df = event_df.loc[
    event_df.groupby(['receptacle_id', 'leg_number'])['actual_depart_datetime_local'].idxmax()
]

In [ ]:
# Set the ULD_id so that the ULD_id is of the last leg (the leg into the US) for all rows no matter what leg it is (eg leg 1 should have uld_id of the last leg)

latest_uld = event_df.sort_values('leg_number').groupby('receptacle_id').last()['uld_id']
event_df['uld_id'] = event_df['receptacle_id'].map(latest_uld)

event_df.rename(columns={'uld_id': 'last_leg_uld_id'}, inplace=True)

In [ ]:
"""
Create and populate Flight Carrier 1, Flight Number 1, Flight Date 1, actual_depart_datetime 1, actual_arrive_datetime 1
Flight Carrier 2, Flight Number 2, Flight Date 2, actual_depart_datetime 2, actual_arrive_datetime 2
To n columns

Need to dynamically create n columns where n is the max number of legs in the table and populate accordingly
Eg if a flight has m legs where m < n, null will be put in for columns m+1, m+2...n
"""


# Find the maximum number of legs in the table (eg some flights can have 2 legs, some can have 3, 4 etc. We want the highest number of legs)
leg_counts = event_df.groupby('receptacle_id')['leg_number'].nunique()
max_legs = leg_counts.max()

for i in range(max_legs):
    event_df[f'Flight Carrier {i+1}'] = None
    event_df[f'Flight Number {i+1}'] = None
    event_df[f'Flight Date {i+1}'] = None
    event_df[f'actual_depart_datetime {i+1}'] = None
    event_df[f'actual_arrive_datetime {i+1}'] = None



# Extract flight_date from actual_depart_datetime_local
event_df['flight_date_from_datetime'] = pd.to_datetime(event_df['actual_depart_datetime_local']).dt.date

# Create a subset with all columns needed for pivoting
pivot_data = event_df[[
    'receptacle_id', 'leg_number',
    'carrier_code', 'flight_number', 'flight_date_from_datetime',
    'actual_depart_datetime_local', 'actual_arrive_datetime_local'
]]

# Pivot the data so each leg becomes a set of fields
pivoted = pivot_data.pivot(index='receptacle_id', columns='leg_number')

# Rename fields to match desired format
pivoted.columns = [
    f"{'Flight Carrier' if col[0] == 'carrier_code' else 'Flight Number' if col[0] == 'flight_number' else 'Flight Date' if col[0] == 'flight_date_from_datetime' else 'actual_depart_datetime' if col[0] == 'actual_depart_datetime_local' else 'actual_arrive_datetime' if col[0] == 'actual_arrive_datetime_local' else col[0]} {col[1]}" for col in pivoted.columns
]

# Reset index to make receptacle_id a field again
pivoted = pivoted.reset_index()

# Merge back with the original DataFrame (dropping duplicates to avoid row explosion)
flight_detail_cols = [col for col in event_df.columns if (
    col.startswith("Flight Carrier") or
    col.startswith("Flight Number") or
    col.startswith("Flight Date") or
    col.startswith("actual_depart_datetime") or
    col.startswith("actual_arrive_datetime")
)]

# Drop duplicates and remove all flight detail columns dynamically
event_df = (
    event_df.drop_duplicates(subset='receptacle_id')
            .drop(columns=flight_detail_cols, errors='ignore')
            .merge(pivoted, on='receptacle_id', how='left')
)


In [78]:
# Removing uncessary fields

# Base columns which will be the same every time
columns_to_keep = [
    "receptacle_id",
    'fct_cgo_gls_mail_cardit_sk',
    'last_leg_uld_id',
    'mail_consign_number'
]

# Add all columns that match the flight detail patterns (for 1...n)
flight_columns = [col for col in event_df.columns if (
    col.startswith("Flight Carrier") or
    col.startswith("Flight Number") or
    col.startswith("Flight Date") or
    col.startswith("actual_depart_datetime") or
    col.startswith("actual_arrive_datetime")
)]

# Combine both sets of columns
final_columns = columns_to_keep + flight_columns

event_df = event_df[final_columns]


## Merging Tables

### Merging Master and Cardit

In [79]:
# Inner join between master_df and cardit_df
master_inner_cardit_df = pd.merge(master_df, cardit_df, on='dim_cgo_cargo_awb_master_sk', how='inner')

# Data that failed to merged from master_df
left_only = pd.merge(master_df, cardit_df, on='dim_cgo_cargo_awb_master_sk', how='left', indicator=True)
master_left_unmatched_cardit_df = left_only[left_only['_merge'] == 'left_only']

# Data that failed to merge from cardit_df
right_only = pd.merge(master_df, cardit_df, on='dim_cgo_cargo_awb_master_sk', how='right', indicator=True)
master_right_unmatched_cardit_df = right_only[right_only['_merge'] == 'right_only']

### Merging Master Cardit and Event

In [80]:
# Inner join between master_inner_cardit_df and event_df
master_cardit_inner_event_df = pd.merge(master_inner_cardit_df, event_df, on='fct_cgo_gls_mail_cardit_sk', how='inner')

# Data that failed to merge from master_inner_cardit_df
left_only = pd.merge(master_inner_cardit_df, event_df, on='fct_cgo_gls_mail_cardit_sk', how='left', indicator=True)
master_cardit_left_unmatched_event_df = left_only[left_only['_merge'] == 'left_only']

# Data that failed to merge from event_df
right_only = pd.merge(master_inner_cardit_df, event_df, on='fct_cgo_gls_mail_cardit_sk', how='right', indicator=True)
master_cardit_right_unmatched_event_df = right_only[right_only['_merge'] == 'right_only']

## Post Merge Processing

In [81]:
# Removing uncessary fields (eg surrogate keys)

master_cardit_inner_event_df.drop(columns=['receptacle_charge_weight', 'fct_cgo_gls_mail_cardit_sk', 'dim_cgo_cargo_awb_master_sk'], inplace=True)

In [82]:
"""
Adding the field Arrival Date: the date of the last actual_arrive_datetime_local of the last leg for each flight
"""

arrival_cols = [col for col in master_cardit_inner_event_df.columns if col.startswith('actual_arrive_datetime')]

if arrival_cols:
    master_cardit_inner_event_df['Arrival Date'] = (
        master_cardit_inner_event_df[arrival_cols]
        .max(axis=1)  # Get the latest datetime across the row
        .dt.date      # Extract just the date
    )
else:
    master_cardit_inner_event_df['Arrival Date'] = pd.NaT  # Fallback if no columns found


In [83]:
# Renaming fields to match CNP template

master_cardit_inner_event_df = master_cardit_inner_event_df.rename(columns={
    'awb_number': 'PAWB',
    'shipment_origin': 'Host Origin Station',
    'shipment_dest': 'Host Destination Station',
    'receptacle_id': 'Receptacle',
    'last_leg_uld_id': 'Arrival ULD number',
    'mail_consign_number': 'CARDIT',
})


In [84]:
""" Reorder columns to match the CNP template"""

base_before_columns = [
    'Receptacle',
    'PAWB',
    'CARDIT',
    'Host Origin Station',
    'Host Destination Station',
]

base_after_columns = [
    'Arrival Date',
    'Arrival ULD number',
]

# Get all leg numbers from the flight-related columns
import re

# Extract all leg numbers present in the DataFrame
leg_numbers = sorted(set(
    int(re.search(r'\d+$', col).group())
    for col in master_cardit_inner_event_df.columns
    if re.search(r'\d+$', col) and (
        col.startswith('Flight Carrier') or
        col.startswith('Flight Number') or
        col.startswith('Flight Date') or
        col.startswith('actual_depart_datetime') or
        col.startswith('actual_arrive_datetime')
    )
))

# Build ordered flight columns
ordered_flight_columns = []
for n in leg_numbers:
    ordered_flight_columns.extend([
        f'Flight Carrier {n}',
        f'Flight Number {n}',
        f'Flight Date {n}',
        f'actual_depart_datetime {n}',
        f'actual_arrive_datetime {n}'
    ])

# Combine and reorder
master_cardit_inner_event_df = master_cardit_inner_event_df[base_before_columns + ordered_flight_columns + base_after_columns]


In [ ]:
# The current dataframe can be exported for general usage. Contains all receptacles, their awb, cardit, o&d, flight details of all legs, depart and arrival of all legs, uld number
# This dataframe is the final processed CX data 
master_cardit_inner_event_df.to_excel('master_cardit_inner_event_df.xlsx', index=False)

## Merging with CNP Data

In [ ]:
# Load CNP data 
cnp_df = pd.read_excel('Sample Data.xlsx', sheet_name='Raw data provided by CNP')

In [87]:
# Formatting CNP data to allow for reading

# Set row 3 as the columns
cnp_df.columns = cnp_df.iloc[3]

# Drop rows 0 to 4 (inclusive)
cnp_df = cnp_df.drop(index=range(0, 5)).reset_index(drop=True)

In [88]:
# Merging master_cardit_inner_event_df (cx data) with CNP data

# Inner join between master_cardit_inner_event_df and cnp_df
cx_inner_cnp_df = pd.merge(master_cardit_inner_event_df, cnp_df, on='Receptacle', how='inner')

# Data that failed to merge from master_cardit_inner_event_df
left_only = pd.merge(master_cardit_inner_event_df, cnp_df, on='Receptacle', how='left', indicator=True)
cx_left_unmatched_cnp_df = left_only[left_only['_merge'] == 'left_only']

# Data that failed to merge from cnp_df
right_only = pd.merge(master_cardit_inner_event_df, cnp_df, on='Receptacle', how='right', indicator=True)
cx_right_unmatched_cnp_df = right_only[right_only['_merge'] == 'right_only']


#### Adding new columns to match output 1 (internal use)
Columns to add: 
    Number of Packet under same receptacle, 
    Tariff amount (using 80% now), 
    A column increasing from 1...n with no name (required in output 1)



In [89]:
# Number of Packets under same receptacle (eg the same receptacle can have mulitple packets)
cx_inner_cnp_df['Number of Packet under same receptacle'] = cx_inner_cnp_df.groupby('Receptacle')['Receptacle'].transform('count')

In [90]:
# Tariff amount

# Ensure the column is numeric (convert if needed)
cx_inner_cnp_df['Customs Declared Value'] = pd.to_numeric(cx_inner_cnp_df['Customs Declared Value'], errors='coerce')

# Calculate Tariff amount and round to 2 decimal places
cx_inner_cnp_df['Tariff amount'] = (cx_inner_cnp_df['Customs Declared Value'] * 0.8).round(2)

In [ ]:
# First column increasing number
cx_inner_cnp_df[''] = range(1, len(cx_inner_cnp_df) + 1)

### Pre Export Processing Ouput 1

In [92]:
# Renaming fields to match CNP template
cx_inner_cnp_df.rename(columns={
    'Receptacle Weight': 'Bag weight',
    'Content': 'Declared content',
    'HS code': 'HS Code',
    'Customs Declared Value': 'Declared Value',
    'Receptacle': 'Receptacle',      
    }, inplace=True)

In [93]:
# Reordering the fields to match CNP template

# Convert all column names to strings
cx_inner_cnp_df.columns = cx_inner_cnp_df.columns.map(str)

# Define start and end columns
start_cols = ['', 'PAWB', 'CARDIT', 'Host Origin Station', 'Host Destination Station']
end_cols = [
    'Arrival Date', 'Arrival ULD number', 
    'Receptacle', 'Bag weight', 'Bag Number',
    'Tracking Number', 'Declared content', 'HS Code',
    'Declared Value', 'Currency', 'Number of Packet under same receptacle', 'Tariff amount'
]

# Add dynamic flight-related columns (can have any number of legs) - Flight carrier 1, 2, 3... needs to be reordered dynamically
flight_cols = [col for col in cx_inner_cnp_df.columns if any(
    str(col).startswith(prefix) for prefix in [
        'Flight Carrier', 'Flight Number', 'Flight Date'
    ]
)]

# Combine and reorder. This merge
new_order = start_cols + flight_cols + end_cols
output_1_df = cx_inner_cnp_df[new_order]


In [94]:
# The output_1_df can be exported for output 1 (internal use)
output_1_df.to_excel('output_1_df.xlsx', index=False)

### Creating Output 2 (CBP) from Output 1

In [95]:
import re

# Ensure column names are strings
df = output_1_df.copy()
df.columns = df.columns.map(str)

# Map Host Destination Station to Arrival Port Code
port_code_map = {
    'JFK': 4701,
    'ORD': 3901,
    'LAX': 2720
}

# If a destination station can't be matched with an arrival port code, fill with 0
df['Arrival Port Code'] = df['Host Destination Station'].map(port_code_map).fillna(0).astype(int)

# Identify all flight leg numbers from column names
flight_leg_nums = sorted(set(
    int(re.search(r'\d+$', col).group())
    for col in df.columns
    if re.search(r'\d+$', col) and (
        col.startswith('Flight Carrier') or col.startswith('Flight Number')
    )
))

# Function to get the highest available flight leg per row
def get_highest_leg_value(row, prefix):
    for leg in reversed(flight_leg_nums):
        col_name = f"{prefix} {leg}"
        if col_name in row and pd.notnull(row[col_name]):
            return row[col_name]
    return None

# Apply function to get Carrier Code and Flight Number
df['Carrier Code'] = df.apply(lambda row: get_highest_leg_value(row, 'Flight Carrier'), axis=1)
df['Flight/Trip Number'] = df.apply(lambda row: get_highest_leg_value(row, 'Flight Number'), axis=1)

# Format Arrival Date and Declared Value
df['Arrival Date'] = pd.to_datetime(df['Arrival Date'], errors='coerce').dt.strftime('%d/%m/%Y')
df['Declared Value (USD)'] = df['Declared Value'].apply(lambda x: f"${x:.2f}" if pd.notnull(x) else "")

# Create the final dataframe
cbp_df = df[['Carrier Code', 'Flight/Trip Number', 'Tracking Number', 'Arrival Port Code', 'Arrival Date', 'Declared Value (USD)']]

In [96]:
# The cbp_df can be exported for output 1 (internal use)
cbp_df.to_excel('cbp_df.xlsx', index=False)